# Librerías

In [43]:
# Librerías
import pandas as pd
import os
import pickle
import configparser
from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.selection import DropFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Configuración inicial

In [44]:
# Configuración inicial
project_path = os.getcwd()
raw_data_path = os.path.join(project_path, "..", "data", "interim", "creditcard_balanced.csv")
artifacts_path = os.path.join(project_path, "..", "artifacts")
processed_data_path = os.path.join(project_path, "..", "data", "processed")
config_path = os.path.join(project_path, "..", "pipeline.cfg")

# Crear carpeta de artefactos si no existe
os.makedirs(artifacts_path, exist_ok=True)
os.makedirs(processed_data_path, exist_ok=True)

In [45]:
# Leer el dataset balanceado
try:
    data = pd.read_csv(raw_data_path)
    print(f"Datos cargados desde {raw_data_path}. Dimensiones: {data.shape}")
except FileNotFoundError:
    raise FileNotFoundError(f"El archivo {raw_data_path} no existe. Verifica la ruta.")

Datos cargados desde c:\Users\hp i7\OneDrive - Universidad Rafael Landivar\Universidad\Galileo\2024\Cuarto Trimestre\Product Development\mini_proyecto_2\mini_proyecto_3\Mini_Proyecto_2\notebooks\..\data\interim\creditcard_balanced.csv. Dimensiones: (1476, 31)


# Cargar archivo de configuración

In [46]:
# Leer configuraciones desde el archivo `pipeline.cfg`
config = configparser.ConfigParser()
if not os.path.exists(config_path):
    raise FileNotFoundError(f"El archivo de configuración {config_path} no existe. Por favor, crea el archivo antes de continuar.")

config.read(config_path)

['c:\\Users\\hp i7\\OneDrive - Universidad Rafael Landivar\\Universidad\\Galileo\\2024\\Cuarto Trimestre\\Product Development\\mini_proyecto_2\\mini_proyecto_3\\Mini_Proyecto_2\\notebooks\\..\\pipeline.cfg']

In [47]:
# Validar secciones requeridas
required_sections = ['GENERAL', 'CONTINUES', 'CATEGORICAL']
missing_sections = [section for section in required_sections if section not in config]
if missing_sections:
    raise ValueError(f"Faltan las siguientes secciones en el archivo de configuración: {missing_sections}")

# Validar columnas

In [48]:
# Validar columnas según configuración
def validate_columns(data, config, section, key):
    """
    Valida que las columnas especificadas en la configuración existan en el dataset.
    Maneja casos en los que las secciones están vacías.

    Args:
        data (pd.DataFrame): El dataset a validar.
        config (ConfigParser): Archivo de configuración cargado.
        section (str): Nombre de la sección en el archivo de configuración.
        key (str): Clave de la sección a validar.

    Returns:
        list: Lista de columnas validadas o vacía si no hay columnas especificadas.
    """
    if key not in config[section] or not config[section][key].strip():
        print(f"No se encontraron columnas para {section} -> {key}.")
        return []
    
    columns = [col.strip() for col in config[section][key].split(',')]
    missing_columns = [col for col in columns if col not in data.columns]
    if missing_columns:
        raise ValueError(f"Las siguientes columnas no existen en el dataset: {missing_columns}")
    return columns


In [49]:
# Validar columnas según configuración
vars_to_drop = validate_columns(data, config, 'GENERAL', 'VARS_TO_DROP')
vars_to_impute_continues = validate_columns(data, config, 'CONTINUES', 'VARS_TO_IMPUTE')
vars_to_impute_categorical = validate_columns(data, config, 'CATEGORICAL', 'VARS_TO_IMPUTE')
ohe_vars = validate_columns(data, config, 'CATEGORICAL', 'OHE_VARS')
freq_enc_vars = validate_columns(data, config, 'CATEGORICAL', 'FREQUENCY_ENC_VARS')


No se encontraron columnas para CONTINUES -> VARS_TO_IMPUTE.
No se encontraron columnas para CATEGORICAL -> VARS_TO_IMPUTE.
No se encontraron columnas para CATEGORICAL -> OHE_VARS.
No se encontraron columnas para CATEGORICAL -> FREQUENCY_ENC_VARS.


# Crear pipeline

In [50]:
# Crear pipeline dinámico
pipeline_steps = []

# Paso para eliminar columnas no relevantes
if vars_to_drop:
    pipeline_steps.append(('delete_features', DropFeatures(features_to_drop=vars_to_drop)))

# Paso para imputación de valores continuos
if vars_to_impute_continues:
    pipeline_steps.append(('mean_imputer', MeanMedianImputer(imputation_method='mean', variables=vars_to_impute_continues)))

# Paso para imputación de valores categóricos
if vars_to_impute_categorical:
    pipeline_steps.append(('categorical_imputer', CategoricalImputer(imputation_method='frequent', variables=vars_to_impute_categorical)))

# Paso para codificación One-Hot
if ohe_vars:
    pipeline_steps.append(('one_hot_encoder', OneHotEncoder(variables=ohe_vars, drop_last=True)))

# Paso para codificación por frecuencia
if freq_enc_vars:
    pipeline_steps.append(('frequency_encoder', CountFrequencyEncoder(encoding_method='count', variables=freq_enc_vars)))

# Paso para escalar características
pipeline_steps.append(('scaling', StandardScaler()))

# Crear el pipeline
pipeline = Pipeline(pipeline_steps)

print("\nPipeline creado dinámicamente con los siguientes pasos:")
for step_name, step in pipeline.steps:
    print(f"- {step_name}: {step.__class__.__name__}")


Pipeline creado dinámicamente con los siguientes pasos:
- delete_features: DropFeatures
- scaling: StandardScaler


# Guardar pipeline

In [51]:
# Guardar el pipeline en la carpeta `artifacts`
pipeline_path = os.path.join(artifacts_path, "base_pipeline.pkl")
try:
    with open(pipeline_path, "wb") as f:
        pickle.dump(pipeline, f)
    print(f"\nPipeline guardado como archivo: {pipeline_path}")
except Exception as e:
    raise ValueError(f"Error al guardar el pipeline: {e}")


Pipeline guardado como archivo: c:\Users\hp i7\OneDrive - Universidad Rafael Landivar\Universidad\Galileo\2024\Cuarto Trimestre\Product Development\mini_proyecto_2\mini_proyecto_3\Mini_Proyecto_2\notebooks\..\artifacts\base_pipeline.pkl


# Generar características

In [52]:
# Ajustar pipeline y generar datos procesados
try:
    # Dividir datos en entrenamiento y prueba
    X = data.drop(columns=[config['GENERAL']['TARGET']])
    y = data[config['GENERAL']['TARGET']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Ajustar el pipeline con los datos de entrenamiento
    pipeline.fit(X_train, y_train)

    # Transformar los conjuntos de entrenamiento y prueba
    X_train_processed = pipeline.transform(X_train)
    X_test_processed = pipeline.transform(X_test)

    # Guardar los datos procesados
    pd.DataFrame(X_train_processed).to_csv(os.path.join(processed_data_path, "X_train.csv"), index=False)
    y_train.to_csv(os.path.join(processed_data_path, "y_train.csv"), index=False)
    pd.DataFrame(X_test_processed).to_csv(os.path.join(processed_data_path, "X_test.csv"), index=False)
    y_test.to_csv(os.path.join(processed_data_path, "y_test.csv"), index=False)

    print("\nConjuntos de datos procesados guardados correctamente.")
except Exception as e:
    raise ValueError(f"Error al procesar los datos: {e}")


Conjuntos de datos procesados guardados correctamente.
